In [4]:
%pip install segmentation-models-pytorch pytorch-lightning albumentations
%pip install --upgrade numpy
%pip install --upgrade matplotlib
%pip install tensorflow opencv-python scikit-learn pillow


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached numpy-2.3.4-cp311-cp311-macosx_14_0_arm64.whl.metadata (62 kB)
Using cached numpy-2.3.4-cp311-cp311-macosx_14_0_arm64.whl (5.4 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.4 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.4 which is in

##### Step 1. Convert YOLOv8 Data to U-Net Segmentation Masks

Summary: Converting the YOLO bounding boxes into U-Net pixel-wise segmentation masks (integer-encoded grayscale images).

In [5]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
import glob

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

print("Core libraries imported successfully!")
print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

Core libraries imported successfully!
TensorFlow version: 2.19.0
NumPy version: 2.1.3


In [6]:
class TrafficSignDataset:
    def __init__(self, data_path, img_size=(256, 256)):
        self.data_path = data_path
        self.img_size = img_size
        self.images = []
        self.masks = []
        
    def find_data_directories(self):
        """Find train/val/test directories"""
        # Check common directory structures
        possible_paths = [
            os.path.join(self.data_path, 'train'),
            os.path.join(self.data_path, 'Train'),
            self.data_path
        ]
        
        for path in possible_paths:
            if os.path.exists(path):
                print(f"Found directory: {path}")
                return path
        raise FileNotFoundError(f"Could not find data directories in {self.data_path}")
    
    def load_yolo_annotations(self, img_path, img_shape):
        """Convert YOLO format to segmentation mask"""
        # Get annotation file path
        annotation_path = img_path.replace('images', 'labels').replace('.jpg', '.txt')
        
        # Create empty mask
        mask = np.zeros(img_shape[:2], dtype=np.uint8)
        
        if os.path.exists(annotation_path):
            try:
                with open(annotation_path, 'r') as f:
                    lines = f.readlines()
                    
                for line in lines:
                    data = line.strip().split()
                    if len(data) >= 5:  # YOLO format: class x_center y_center width height
                        x_center, y_center, width, height = map(float, data[1:5])
                        
                        # Convert normalized coordinates to pixel coordinates
                        h, w = img_shape[:2]
                        x_center *= w
                        y_center *= h
                        width *= w
                        height *= h
                        
                        # Calculate bounding box coordinates
                        x1 = max(0, int(x_center - width/2))
                        y1 = max(0, int(y_center - height/2))
                        x2 = min(w, int(x_center + width/2))
                        y2 = min(h, int(y_center + height/2))
                        
                        # Create binary mask for traffic signs
                        if x2 > x1 and y2 > y1:
                            mask[y1:y2, x1:x2] = 1
            except Exception as e:
                print(f"Error processing annotation {annotation_path}: {e}")
                    
        return mask
    
    def load_data(self, max_samples=None):
        """Load images and create corresponding masks"""
        try:
            base_path = self.find_data_directories()
        except FileNotFoundError:
            print(f"Warning: Could not find standard directories in {self.data_path}")
            print("Trying to find images in the root directory...")
            base_path = self.data_path
        
        # Find all image files
        image_files = []
        search_paths = [
            os.path.join(base_path, 'images', '*.jpg'),
            os.path.join(base_path, 'images', '*.png'),
            os.path.join(base_path, '*.jpg'),
            os.path.join(base_path, '*.png')
        ]
        
        for path in search_paths:
            image_files.extend(glob.glob(path))
        
        if not image_files:
            raise FileNotFoundError(f"No image files found in {self.data_path}")
        
        for i, img_file in enumerate(image_files):
        
            print(f"Found {len(image_files)} image files")
        
        for i, img_file in enumerate(image_files):
            if i % 100 == 0:
                print(f"Processing image {i}/{len(image_files)}")
                
            try:
                # Load image
                image = cv2.imread(img_file)
                if image is None:
                    continue
                    
                # Preprocess image
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, self.img_size)
                image = image.astype(np.float32) / 255.0
                
                # Create mask
                original_img = cv2.imread(img_file)
                mask = self.load_yolo_annotations(img_file, original_img.shape)
                mask = cv2.resize(mask, self.img_size, interpolation=cv2.INTER_NEAREST)
                mask = mask.astype(np.float32)
                
                self.images.append(image)
                self.masks.append(mask)
                
            except Exception as e:
                print(f"Error processing {img_file}: {e}")
                continue
            
        if len(self.images) == 0:
            raise ValueError("No images were successfully loaded!")
            
        self.images = np.array(self.images)
        self.masks = np.array(self.masks)
        self.masks = np.expand_dims(self.masks, -1)  # Add channel dimension
        
        print(f"Successfully loaded {len(self.images)} images and masks")
        print(f"Images shape: {self.images.shape}")
        print(f"Masks shape: {self.masks.shape}")
        
        return self.images, self.masks

In [7]:
def create_simple_unet(input_size=(256, 256, 3)):
    """Create a simpler U-Net model for faster testing"""
    inputs = keras.Input(input_size)
    
    # Downsample
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x1 = x
    x = layers.MaxPooling2D(2)(x)
    
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x2 = x
    x = layers.MaxPooling2D(2)(x)
    
    # Bottleneck
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(128, 3, activation='relu', padding='same')(x)
    
    # Upsample
    x = layers.Conv2DTranspose(64, 2, strides=2, padding='same')(x)
    x = layers.concatenate([x, x2])
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(64, 3, activation='relu', padding='same')(x)
    
    x = layers.Conv2DTranspose(32, 2, strides=2, padding='same')(x)
    x = layers.concatenate([x, x1])
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    x = layers.Conv2D(32, 3, activation='relu', padding='same')(x)
    
    # Output
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    return model

In [8]:
def calculate_iou(y_true, y_pred):
    """Calculate Intersection over Union"""
    y_pred_binary = (y_pred > 0.5).astype(np.float32)
    intersection = np.sum(y_true * y_pred_binary)
    union = np.sum(y_true) + np.sum(y_pred_binary) - intersection
    return intersection / (union + 1e-7)

def calculate_metrics(y_true, y_pred):
    """Calculate precision, recall, and F1-score"""
    y_true_flat = y_true.flatten()
    y_pred_flat = (y_pred.flatten() > 0.5).astype(np.int32)
    
    # Handle edge cases
    if np.sum(y_true_flat) == 0 and np.sum(y_pred_flat) == 0:
        return 1.0, 1.0, 1.0
    
    precision = precision_score(y_true_flat, y_pred_flat, zero_division=0)
    recall = recall_score(y_true_flat, y_pred_flat, zero_division=0)
    f1 = f1_score(y_true_flat, y_pred_flat, zero_division=0)
    
    return precision, recall, f1

In [9]:
def train_traffic_sign_detector(data_path=".", max_samples=50, epochs=5, batch_size=4):
    """Main training function"""
    print("=== Traffic Sign Detection System ===")
    print("Loading data...")
    
    try:
        # Load dataset
        # dataset = TrafficSignDataset(data_path)
        # X, y = dataset.load_data(max_samples=max_samples)
        dataset = TrafficSignDataset(data_path)
        X, y = dataset.load_data()  # No max_samples limit
    
        
        print(f"Loaded {X.shape[0]} samples")
        
        # Split data
        X_train, X_val, y_train, y_val = train_test_split(
            X, y, test_size=0.2, random_state=42
        )
        
        print(f"Training on {X_train.shape[0]} samples")
        print(f"Validating on {X_val.shape[0]} samples")
        
        # Create model
        print("Creating U-Net model...")
        model = create_simple_unet()
        
        # Compile model
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        
        print("Model architecture:")
        print(f"Input: {model.input_shape}")
        print(f"Output: {model.output_shape}")
        
        # Train model
        print("Starting training...")
        history = model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )
        
        # Evaluate
        print("Evaluating model...")
        y_pred = model.predict(X_val)
        
        iou = calculate_iou(y_val, y_pred)
        precision, recall, f1 = calculate_metrics(y_val, y_pred)
        
        print("\n" + "="*60)
        print("EVALUATION RESULTS")
        print("="*60)
        print(f"IoU (Intersection over Union): {iou:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print("="*60)
        
        # Save model
        model.save('traffic_sign_unet.h5')
        print("Model saved as 'traffic_sign_unet.h5'")
        
        return model, history
        
    except Exception as e:
        print(f"Error during training: {e}")
        print("Please check your data path and dataset structure")
        return None, None

In [10]:
def quick_system_test():
    """Test if the basic system works"""
    print("=== Running System Test ===")
    
    # Test 1: Model creation
    print("1. Testing model creation...")
    try:
        model = create_simple_unet()
        print("✓ Model created successfully")
    except Exception as e:
        print(f"✗ Model creation failed: {e}")
        return False
    
    # Test 2: Data loading with dummy data
    print("2. Testing data pipeline...")
    try:
        # Create dummy data
        dummy_images = np.random.random((2, 256, 256, 3)).astype(np.float32)
        dummy_masks = np.random.randint(0, 2, (2, 256, 256, 1)).astype(np.float32)
        
        # Test prediction
        pred = model.predict(dummy_images, verbose=0)
        print("✓ Data pipeline test passed")
    except Exception as e:
        print(f"✗ Data pipeline test failed: {e}")
        return False
    
    # Test 3: Metrics calculation
    print("3. Testing metrics calculation...")
    try:
        iou = calculate_iou(dummy_masks, pred)
        precision, recall, f1 = calculate_metrics(dummy_masks, pred)
        print("✓ Metrics calculation test passed")
    except Exception as e:
        print(f"✗ Metrics calculation failed: {e}")
        return False
    
    print("✓ All system tests passed!")
    return True

# Run the system test first
if quick_system_test():
    print("\nSystem is ready for training!")
    
    # Now run the actual training with a small sample
    data_path = "data/car"  # Change this to your actual data path
    
    print(f"\nStarting training with data from: {data_path}")
    # model, history = train_traffic_sign_detector(
    #     data_path=data_path,
    #     max_samples=50,    # Small sample for testing
    #     epochs=5,          # Few epochs for testing
    #     batch_size=4       # Small batch size
    # )
    # Remove test_mode and run full training
    print("Starting FULL dataset training...")
    model, history = train_traffic_sign_detector(
        data_path=data_path,
        epochs=50,          # More epochs for full training
        batch_size=16       # Larger batch size
    )
else:
    print("System test failed. Please check your environment.")

=== Running System Test ===
1. Testing model creation...
✓ Model created successfully
2. Testing data pipeline...
✓ Data pipeline test passed
3. Testing metrics calculation...
✓ Metrics calculation test passed
✓ All system tests passed!

System is ready for training!

Starting training with data from: data/car
Starting FULL dataset training...
=== Traffic Sign Detection System ===
Loading data...
Found directory: data/car/train
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image files
Found 3530 image


EVALUATION RESULTS
IoU (Intersection over Union): 0.8405
Precision: 0.8876
Recall: 0.9406
F1-Score: 0.9133
Model saved as 'traffic_sign_unet.h5'
